In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,AveragePooling2D,Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from joblib import dump, load
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model

from imutils import face_utils
import argparse
import imutils
import dlib
from datetime import datetime

import os
import cv2



In [2]:
def reshape_data(input_data):
    nsamples, nx, ny = input_data.shape
    return input_data.reshape((nsamples, nx*ny))

def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)

# inicijalizaclija dlib detektora (HOG)
detector = dlib.get_frontal_face_detector()
# ucitavanje pretreniranog modela za prepoznavanje karakteristicnih tacaka
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [3]:
X = []
Y = []
for image_name in os.listdir('dataset/without_mask'):
            resized = cv2.resize(cv2.imread('dataset/without_mask/' + image_name), (64,64))
            X.append(resized)
            Y.append(0)
            
            
for image_name in os.listdir('dataset/with_mask'):
            resized = cv2.resize(cv2.imread('dataset/with_mask/' + image_name), (64,64))
            X.append(resized)
            Y.append(1)


In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2)

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [3]:
try:
    cnn = load_model('model/cnn.h5')
    print('ucitan model')
except Exception as e:
    cnn = None
    
if cnn == None:

    cnn = Sequential()
    cnn.add(Conv2D(filters = 32, kernel_size =(3,3), 
                   activation = 'relu',input_shape = (64,64,3)))
    cnn.add(Conv2D(filters = 32, kernel_size =(3,3), 
                   activation = 'relu'))
    cnn.add(MaxPooling2D(2,2))
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(filters = 64, kernel_size =(3,3), activation = 'relu'))
    cnn.add(Conv2D(filters = 64, kernel_size =(3,3), 
                   activation = 'relu'))
    cnn.add(MaxPooling2D(2,2))
    cnn.add(Dropout(0.2))

    cnn.add(Flatten())

    cnn.add(Dense(units = 512, activation = 'relu'))
    cnn.add(Dense(units = 512, activation = 'relu'))
    cnn.add(Dense(units = 1, activation = 'sigmoid'))

    cnn.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

    X_train = X_train/255
    X_test = X_test/255

    X_train, y_train = shuffle(X_train, y_train)

    hist = cnn.fit(X_train,y_train,epochs = 10,batch_size = 64,verbose = 1, validation_split = 0.2)
    
    model = save_model(cnn, 'model/cnn.h5')


ucitan model


In [6]:
pred = cnn.predict(X_test)

In [7]:
pred[pred>0.5]=1
pred[pred!=1]=0

In [8]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,pred)
cm


array([[84,  0],
       [ 1, 72]], dtype=int64)

In [9]:
from sklearn.metrics import f1_score
score=f1_score(y_test, pred, average='macro')
score

0.9935931442562742

In [7]:
cap = cv2.VideoCapture(0)


nbins = 9  # broj binova
cell_size = (8, 8)  # broj piksela po celiji
block_size = (3, 3)  # broj celija po bloku

dim = (244, 244)


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    features = [];
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for (b, rect) in enumerate(rects):

        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        (xosa, yosa, w, h) = face_utils.rect_to_bb(rect)
        face = frame[yosa:yosa + h, xosa:xosa + w]
           
    resized = cv2.resize(face, (64,64))

    features.append(resized)

    x = np.array(features)
    x = x/255
        
    pred = cnn.predict(x)

    if pred < 0.5:
        cv2.rectangle(frame, (xosa, yosa), (xosa + w, yosa + h), (0, 0, 255), 2)
        cv2.putText(frame, "without mask", (xosa, yosa),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    else:
        cv2.rectangle(frame, (xosa, yosa), (xosa + w, yosa + h), (0, 255, 0), 2)
        cv2.putText(frame, "with mask", (xosa, yosa),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [5]:
file=open('dataset/annotations.csv')
data = file.read()
lines = data.split('\n')
facial_expression=[]
br_tacnih=0
ukupno=0
before = datetime.now()

for index, line in enumerate(lines):
    cols = line.split(',')
    if cols and cols[0] == '':
        continue
    cols[0] = cols[0].replace('\r', '')
    cols[1] = cols[1].replace('\r', '')

    features = [];

    img = cv2.imread('dataset/test/' + cols[0])
           
    resized = cv2.resize(img, (64,64))
    features.append(resized)

    x = np.array(features)
    x = x/255

    pred = cnn.predict(x)
    

    predikcija=1
    if pred < 0.5:
        predikcija=0

#     print(cols[1])
#     print(predikcija)
#     print(cols[0])
#     print('---------------')

    ukupno+=1
    if int(predikcija) == int(cols[1]):
        br_tacnih+=1
        
after = datetime.now()
    
    
print('Tacnost: '+ str(br_tacnih/ukupno*100))
print('Ukupno vreme: '+ str(after-before))

Tacnost: 93.75
Ukupno vreme: 0:00:00.682539
